## CNN での画像分類

このコードは、PyTorchを使用してCIFAR-10データセットに対する単純な畳み込みニューラルネットワーク（CNN）モデルをトレーニングし、評価するプロセスを示しています。CIFAR-10は、10の異なるカテゴリ（飛行機、自動車、鳥、猫など）に属する60,000枚の小さなカラー画像（32x32ピクセル）を含む一般的なデータセットです。

Google Colab での実行方法は [トップページ](https://github.com/shizoda/education/tree/main) に示しています。

In [ ]:
# PyTorch 関連のライブラリをインポートします
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# GPU が利用可能であることを確認
if not torch.cuda.is_available():
    print("GPU is not available. Please change runtime type to 'GPU' in the Runtime menu.")

# 1. データをロードする
#     CIFAR-10データセットは、トレーニングとテストのためにそれぞれダウンロードされます。
#    transforms.Compose は、データに適用する一連の前処理手順を定義します。この例では、画像をPyTorchテンソルに変換し、正規化を行います。
#    DataLoader は、データのバッチ処理、シャッフル、多重プロセスによるデータの読み込みを扱います。

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 画像を表示するためのヘルパー関数
def imshow(img):
    img = img / 2 + 0.5     # 正規化を元に戻す
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# DataLoader から画像を取得して表示する関数
def show_images(dataloader, num_images):
    dataiter = iter(dataloader)
    images, labels = next(dataiter)  # バッチを一つ取得

    # 画像を表示
    imshow(torchvision.utils.make_grid(images[:num_images]))
    # 正解ラベルを表示
    print(' '.join(f'{classes[labels[j]]:5s}' for j in range(num_images)))

# 使用例：trainloader から 4 枚の画像を表示
show_images(trainloader, 4)


In [ ]:
# 2. CNNモデルを定義する
# Net クラスは、CNNモデルのアーキテクチャを定義します。
# モデルは畳み込み層（nn.Conv2d）、プーリング層（nn.MaxPool2d）、完全に接続された層（nn.Linear）から構成されています。
# forward メソッドは、ネットワークを通じて入力がどのように進むかを定義します

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# インスタンス化。Net クラス（設計図）のオブジェクト（実体）を net とする
net = Net()

# 3. 損失関数とオプティマイザを定義する
#     損失関数（nn.CrossEntropyLoss）は、モデルの予測と実際のラベル間の差異を測定します。
#    最適化アルゴリズム（optim.SGD）は、モデルのパラメータを調整するために使用されます。

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# 4. ネットワークをトレーニングする
#     トレーニングループは、データセットを複数回（エポック）処理します。
#    各イテレーションで、モデルは入力データに対する予測を行い、損失を計算し、勾配を逆伝播させてパラメータを更新します。

for epoch in range(2):  # データセットを複数回ループ
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # 2000ミニバッチごとに出力
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
# 5. ネットワークのテスト
#     テストフェーズでは、訓練されたモデルを使用してテストデータセット上でのパフォーマンスを評価します。
#    最終的な精度（正確に分類された画像の割合）が計算され、出力されます。

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
# テストデータの画像を表示し、正解と推定値を表示する関数
def visualize_test_predictions(start_idx, end_idx):
    testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                             shuffle=False, num_workers=2)
    images, labels = zip(*[(data[0], data[1]) for i, data in enumerate(testloader) if start_idx <= i < end_idx])

    # モデルの予測
    predicted_labels = []
    with torch.no_grad():
        for image in images:
            outputs = net(image)
            _, predicted = torch.max(outputs, 1)
            predicted_labels.append(predicted.item())

    # 画像とラベルを表示
    for i in range(len(images)):
        print(f"Image {start_idx + i}")
        imshow(torchvision.utils.make_grid(images[i]))
        print(f"Correct label: {classes[labels[i]]}")
        print(f"Predicted label: {classes[predicted_labels[i]]}\n")

# 例: インデックス範囲 0 から 5 の画像を表示
visualize_test_predictions(0, 5)
